## Encrypt data with AES

the receiver is not able to detect if the ciphertext (i.e., the encrypted data) was modified while in transit. To address that risk, we also attach a MAC authentication tag (HMAC with SHA256), made with a second key.

In [1]:
!pip install pycryptodome

In [2]:
!pip install pycryptodome-test-vectors

In [3]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import HMAC, SHA256


- **`AES`**: A symmetric encryption algorithm used to encrypt and decrypt data.
- **`get_random_bytes`**: Generates random bytes for keys or initialization vectors.
- **`HMAC`**: Provides data integrity and authenticity using a hash function and a secret key.
- **`SHA256`**: A cryptographic hash function that produces a 256-bit hash value.

In [7]:
input_data = 'PyCryptodome is a self-contained Python package of low-level cryptographic primitives'

data = input_data.encode()

# generate random 16 bit data
aes_key = get_random_bytes(16)  
hmac_key = get_random_bytes(16) 

# Convert the normal Input into the cipher text(Encrypted)
cipher = AES.new(aes_key, AES.MODE_CTR)
ciphertext = cipher.encrypt(data)

# It generate Hash Key 
hmac = HMAC.new(hmac_key, digestmod=SHA256)
tag = hmac.update(cipher.nonce + ciphertext).digest()

# All Data Store into a `Encypted.bin` file
with open("encrypted.bin", "wb") as f:
    f.write(tag)
    f.write(cipher.nonce)
    f.write(ciphertext)
    
aes_key, hmac_key, ciphertext, hmac

(b'\t\x07\xa5\x01\xe5\xbaD\xc5\x8c\xc2<\xd7\xefE\xe6\xdb',
 b'\xdc;\x0b\x87\xbe\xd4\xc1Co\xe0\x9a\xc8\xb4\xcd\xf6\xb7',
 b'\x1bxx=6S\x91\x94\x19\xb3\xe6\xf7$\x97\xa8\xa8e\xaf\xa9:\x04>\x9f\xf0\xf2%MGD\xb5\x013f\x97\xf3\xb1\xefZE\x9d#S\x8f\xb4\x1e\xad\x0c\xfc hd\x97\x9fi\xb7"fA\x8cX\xc2C\xbfu\xa8\x86\xf8-\xc3t\xe1t\xf2\xaa\xff\xb7i\x15\x14\x14\xb6^lil',
 <Crypto.Hash.HMAC.HMAC at 0x12393506cf0>)

At the other end, the receiver can securely load the piece of data back (if they know the two keys!). Note that the code generates a ValueError exception when tampering is detected.

In [8]:
import sys

with open("encrypted.bin", "rb") as f:
    tag = f.read(32)
    nonce = f.read(8)
    ciphertext = f.read()

try:
    hmac = HMAC.new(hmac_key, digestmod=SHA256)
    tag = hmac.update(nonce + ciphertext).verify(tag)
except ValueError:
    print("The message was modified!")
    sys.exit(1)

cipher = AES.new(aes_key, AES.MODE_CTR, nonce=nonce)
message = cipher.decrypt(ciphertext)
print("Message:", message.decode())

Message: PyCryptodome is a self-contained Python package of low-level cryptographic primitives


## PyCryptodome is a self-contained Python package of low-level cryptographic primitives

The code in the previous section contains three subtle but important design decisions: the nonce of the cipher is authenticated, the authentication is performed after encryption, and encryption and authentication use two uncorrelated keys. It is not easy to securely combine cryptographic primitives, so more modern cryptographic cipher modes have been created such as, the OCB mode (see also other authenticated encryption modes like EAX, GCM, CCM, SIV).

In [24]:
data_obc = input_data.encode()

aes_key_obc = get_random_bytes(16)

cipher_obc = AES.new(aes_key_obc, AES.MODE_OCB)
ciphertext_obc, tag_obc = cipher_obc.encrypt_and_digest(data_obc)
assert len(cipher_obc.nonce) == 15

with open("encrypted_obc.bin", "wb") as f:
    f.write(tag_obc)
    f.write(cipher_obc.nonce)
    f.write(ciphertext_obc)
    
cipher_obc, ciphertext_obc, aes_key_obc

(<Crypto.Cipher._mode_ocb.OcbMode at 0x123943c6450>,
 b"pI\x96\xf5I4\tt\n\xc2\xd0\xcc$\xbc[\x98\xac@\x08\xeb\xd3\xdd\xacR\xb0\xa6\xe6\xb3W\xb8]6\xb7\x87wu\x07\x01\xac\x9f\x19\x0b\xa0\xc8U\xd6\x8b\x0c\xee\xa4O_\x13\x89\xd6\x19\x15\x89-\x98\xb9,4\x86'q\xcf\xc2\xd4\xad\xbc\xe3\xda\x1f\xd5_#~\\\xee\x96\x07\xf7O\xfe",
 b"_',o|\xab\x82\xd0X\xbdX\x85\xf6\xce\x98O")

Decryption

In [27]:
with open("encrypted_obc.bin", "rb") as f:
    tag_obc_decrypt = f.read(16)  # Read 16 bytes for the tag
    nonce_obc_decrypt = f.read(15)  # Read 15 bytes for the nonce
    ciphertext_obc_decrypt = f.read()  # Read the remaining ciphertext

# Decrypt using the same key and nonce from the file
cipher_obc_decrypt = AES.new(aes_key_obc, AES.MODE_OCB, nonce=nonce_obc_decrypt)

try:
    # Decrypt and verify the message
    message_obc_decrypt = cipher_obc_decrypt.decrypt_and_verify(ciphertext_obc_decrypt, tag_obc_decrypt)
except ValueError:
    print("The message was modified!")
    sys.exit(1)

# If successful, print the decrypted message
print("Message:", message_obc_decrypt.decode())


Message: PyCryptodome is a self-contained Python package of low-level cryptographic primitives


## Generate an RSA key

The following code generates a new RSA key pair (secret) and saves it into a file, protected by a password. We use the scrypt key derivation function to thwart dictionary attacks. At the end, the code prints our the RSA public key in ASCII/PEM format:

In [29]:
from Crypto.PublicKey import RSA

secret_code = input_data
key = RSA.generate(2048)
encrypted_key = key.export_key(passphrase=secret_code, pkcs=8,
                              protection="scryptAndAES128-CBC",
                              prot_params={'iteration_count':131072})

with open("rsa_key.bin", "wb") as f:
    f.write(encrypted_key)

print(key.publickey().export_key())

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAq8F07UHVm6wyWSjTq6lX\n1xGsVy4NR1V24XQ8RQ1Q4dirsWnrC6YB3XJqR6WnbiD3mykK2BAU28K1Ol7fWSjn\nRukLeXdH/BLrPBx2tW0ACC8nuoK06CEqX0sjzcef0228cuMwIq3C73JsENMVMu6B\n1GVyE8xRdOovHcoTpVMt7q/sOm3srnKqT9y3gcO8yhgzsv+6ttmWNZZzQzjFkzuO\n6yFUKLXPfGvDqAywqhToIDiZhjP1IY/MNUE/HrBzo6sgW1SHVnNQ2hZTyaVOdEEJ\nZ/XCaeHnE9dxaADWG/ANe4Cywq5qT4Wh7icF4QNqpjGW/7e4oNYJBAWtslN12AKw\nYwIDAQAB\n-----END PUBLIC KEY-----'


This code generates an RSA key pair, encrypts the private key using a passphrase, and then writes the encrypted private key to a file. Here's a breakdown of each part:

1. **`from Crypto.PublicKey import RSA`**:
   - Imports the `RSA` module from `pycryptodome` to generate RSA public/private key pairs.

2. **`secret_code = "Unguessable"`**:
   - Defines a passphrase (`secret_code`) that will be used to encrypt the private key. In this case, the passphrase is `"Unguessable"`, but this can be any string.

3. **`key = RSA.generate(2048)`**:
   - Generates a new 2048-bit RSA key pair (both public and private keys). RSA is an asymmetric encryption algorithm commonly used for secure data transmission and digital signatures.

4. **`encrypted_key = key.export_key(...)`**:
   - Exports the private key with encryption using the following options:
     - `passphrase=secret_code`: The private key is encrypted using the passphrase `"Unguessable"`.
     - `pkcs=8`: Specifies that the key will be exported using the **PKCS#8** format, which is a standard for private key information.
     - `protection="scryptAndAES128-CBC"`: Uses the **scrypt** key derivation function combined with **AES-128** encryption in **CBC mode** to secure the private key.
     - `prot_params={'iteration_count':131072}`: This sets the number of iterations for the key derivation function (scrypt), making brute-force attacks harder. In this case, it's set to 131,072 iterations.

5. **`with open("rsa_key.bin", "wb") as f:`**:
   - Opens a binary file named `"rsa_key.bin"` for writing.

6. **`f.write(encrypted_key)`**:
   - Writes the encrypted private key to the file `"rsa_key.bin"`. This ensures that the private key is securely stored and can only be decrypted using the provided passphrase.

7. **`print(key.publickey().export_key())`**:
   - Extracts the public key from the RSA key pair (`key.publickey()`) and prints it in the default PEM format. The public key is not encrypted and can be shared freely since it is used for encryption and signature verification, not decryption.

### Summary:
- The code generates an RSA key pair, encrypts the private key using a passphrase and AES encryption, and writes the encrypted private key to a file.
- It also prints the public key, which can be shared openly for encryption or verification purposes.

This is useful for securely storing private keys and ensuring that only authorized users with the correct passphrase can access the key for decryption or signing operations.

In [30]:
secret_code = input_data
encoded_key = open("rsa_key.bin", "rb").read()
key = RSA.import_key(encoded_key, passphrase=secret_code)

print(key.publickey().export_key())

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAq8F07UHVm6wyWSjTq6lX\n1xGsVy4NR1V24XQ8RQ1Q4dirsWnrC6YB3XJqR6WnbiD3mykK2BAU28K1Ol7fWSjn\nRukLeXdH/BLrPBx2tW0ACC8nuoK06CEqX0sjzcef0228cuMwIq3C73JsENMVMu6B\n1GVyE8xRdOovHcoTpVMt7q/sOm3srnKqT9y3gcO8yhgzsv+6ttmWNZZzQzjFkzuO\n6yFUKLXPfGvDqAywqhToIDiZhjP1IY/MNUE/HrBzo6sgW1SHVnNQ2hZTyaVOdEEJ\nZ/XCaeHnE9dxaADWG/ANe4Cywq5qT4Wh7icF4QNqpjGW/7e4oNYJBAWtslN12AKw\nYwIDAQAB\n-----END PUBLIC KEY-----'


## Generate public key and private key

The following code generates public key stored in receiver.pem and private key stored in private.pem. These files will be used in the examples below. Every time, it generates different public key and private key pair.

In [31]:
key = RSA.generate(2048)
private_key = key.export_key()
with open("private.pem", "wb") as f:
    f.write(private_key)

public_key = key.publickey().export_key()
with open("receiver.pem", "wb") as f:
    f.write(public_key)

## Encrypt data with RSA

The following code encrypts a piece of data for a receiver we have the RSA public key of. The RSA public key is stored in a file called receiver.pem.

Since we want to be able to encrypt an arbitrary amount of data, we use a hybrid encryption scheme. We use RSA with PKCS#1 OAEP for asymmetric encryption of an AES session key. The session key can then be used to encrypt all the actual data.

As in the first example, we use the EAX mode to allow detection of unauthorized modifications.

In [34]:
from Crypto.Cipher import PKCS1_OAEP

In [35]:
data = input_data.encode("utf-8")

recipient_key = RSA.import_key(open("receiver.pem").read())
session_key = get_random_bytes(16)

# Encrypt the session key with the public RSA key

cipher_rsa = PKCS1_OAEP.new(recipient_key)
enc_session_key = cipher_rsa.encrypt(session_key)

# Encrypt the data with the AES session key

cipher_aes = AES.new(session_key, AES.MODE_EAX)
ciphertext, tag = cipher_aes.encrypt_and_digest(data)

with open("encrypted_data.bin", "wb") as f:
    f.write(enc_session_key)
    f.write(cipher_aes.nonce)
    f.write(tag)
    f.write(ciphertext)

The receiver has the private RSA key. They will use it to decrypt the session key first, and with that the rest of the file:

In [36]:
private_key = RSA.import_key(open("private.pem").read())

with open("encrypted_data.bin", "rb") as f:
    enc_session_key = f.read(private_key.size_in_bytes())
    nonce = f.read(16)
    tag = f.read(16)
    ciphertext = f.read()

# Decrypt the session key with the private RSA key
cipher_rsa = PKCS1_OAEP.new(private_key)
session_key = cipher_rsa.decrypt(enc_session_key)

# Decrypt the data with the AES session key
cipher_aes = AES.new(session_key, AES.MODE_EAX, nonce)
data = cipher_aes.decrypt_and_verify(ciphertext, tag)
print(data.decode("utf-8"))

PyCryptodome is a self-contained Python package of low-level cryptographic primitives
